## ニューラルネットワークの実装（分類）

### PyTorch Lightning による学習ループの簡略化

In [1]:
# ライブラリのインストール
!pip install pytorch_lightning

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.3.1


ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [48]:
!pip install pytorch_lightning --ignore-installed PyYAML --user

  Using cached pytorch_lightning-1.9.0-py3-none-any.whl (825 kB)
  Using cached PyYAML-6.0-cp38-cp38-win_amd64.whl (155 kB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached packaging-23.0-py3-none-any.whl (42 kB)
  Using cached fsspec-2023.1.0-py3-none-any.whl (143 kB)
  Using cached numpy-1.24.1-cp38-cp38-win_amd64.whl (14.9 MB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Using cached torch-1.13.1-cp38-cp38-win_amd64.whl (162.6 MB)
  Using cached lightning_utilities-0.5.0-py3-none-any.whl (18 kB)
  Using cached torchmetrics-0.11.0-py3-none-any.whl (512 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
  Using cached aiohttp-3.8.3-cp38-cp38-win_amd64.whl (324 kB)
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Using cached frozenlist-1.3.3-cp38-cp38-win_amd64.whl (34 kB)
  Using cached yarl-1.8.2-cp38-cp38-win_amd64.whl (56 kB)
  Using cached attrs-22.2.0-py3-none-any.whl (60 kB)
  Using cached async_timeout-4.0.2-py3-n

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

scipy 1.7.0 requires numpy<1.23.0,>=1.16.5, but you'll have numpy 1.24.1 which is incompatible.


### データセットの準備

In [2]:
import numpy as np
import pandas as pd

In [5]:
# データの読み込み（df: data frame）
df = pd.read_csv('./data/wine_class.csv')

In [6]:
# データの表示（先頭の5件）
df.head()

,Class,Alcohol,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Color intensity,Hue,Proline
0,1,14.23,2.43,15.6,127,2.80,3.06,0.28,5.64,1.04,1065
1,1,13.20,2.14,11.2,100,2.65,2.76,0.26,4.38,1.05,1050
2,1,13.16,2.67,18.6,101,2.80,3.24,0.30,5.68,1.03,1185
3,1,14.37,2.50,16.8,113,3.85,3.49,0.24,7.80,0.86,1480
4,1,13.24,2.87,21.0,118,2.80,2.69,0.39,4.32,1.04,735


### 入力変数と目的変数に切り分け

In [7]:
np.unique(df['Class'], return_counts=True)

(array([1, 2, 3], dtype=int64), array([59, 71, 48], dtype=int64))

In [35]:
x = df.drop('Class', axis=1)
t = df['Class']

In [9]:
# 表示して確認
x.head(3)

,Alcohol,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Color intensity,Hue,Proline
0,14.23,2.43,15.6,127,2.80,3.06,0.28,5.64,1.04,1065
1,13.20,2.14,11.2,100,2.65,2.76,0.26,4.38,1.05,1050
2,13.16,2.67,18.6,101,2.80,3.24,0.30,5.68,1.03,1185


In [10]:
# サイズの確認
x.shape, t.shape

((178, 10), (178,))

In [30]:
type(x), type(t)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

### tensor に変換

In [14]:
# pandas.core.frame.DataFrame、 pandas.core.series.Series 型から 
# Tensor 型への直接変換ができないため、一度 NumPy の形式に変換
# NumPy の形式に変換するには、.values を使用

In [15]:
import torch

In [36]:
# Tensor 形式へ変換
x = torch.tensor(x.values, dtype=torch.float32)
t = torch.tensor(t.values, dtype=torch.int64)

In [37]:
# 分類の場合にはラベルが 0 から始まらなければならない
# 1, 2, 3 → 0, 1, 2 と変換
t

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [38]:
# ラベルを 0 から始める
t = t - 1 
t

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### dataset にまとめる

In [39]:
# PyTorch では x と t をひとつにまとめることが一般的
# TensorDataset を使う

# 入力変数と目的変数をまとめて、ひとつのオブジェクト dataset に変換
dataset = torch.utils.data.TensorDataset(x, t)
dataset

In [40]:
# (入力変数, 目的変数) のようにタプルで格納されている
dataset[0]

(tensor([1.4230e+01, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00, 3.0600e+00,
         2.8000e-01, 5.6400e+00, 1.0400e+00, 1.0650e+03]),
 tensor(0))

### 学習用データ、検証用データ、テスト用データに分割

In [41]:
# 前章では設定した DataLoader の設定は 
# PyTorch Lightning 側で用意されているため必要なし
# こちらでは、学習用、検証用、テスト用のデータセットの分割だけ行う

In [42]:
# 各データセットのサンプル数を決定
# train : val : test = 60% : 20% : 20%
n_train = int(len(dataset) * 0.6)
n_val = int((len(dataset) - n_train) * 0.5)
n_test = len(dataset) - n_train - n_val

In [43]:
# サンプル数の確認
n_train, n_val, n_test

(106, 36, 36)

In [45]:
# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

### PyTorch Lightning によるモデルと学習手順の定義

In [46]:
# モデルを定義する際のクラスは nn.Module を継承していたが、
# この点を PyTorch Lightning の LightningModule を継承
# まずは検証データとテストデータを抜いた学習データのみに対する最小限のクラスを設計

In [49]:
import pytorch_lightning as pl

In [50]:
# バージョンの確認
pl.__version__

'1.9.0'

In [51]:
import torch.nn as nn
import torch.nn.functional as F

In [53]:
class Net(pl.LightningModule):
    # nn -> pl: バッチサイズ等を引数に指定する
    def __init__(self, input_size=10, hidden_size=5, output_size=3, batch_size=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size

    # nn -> pl: 変更なし
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

    # nn -> pl: 目的関数の設定
    def lossfun(self, y, t):
        return F.cross_entropy(y, t)
    
    # nn -> pl: optimizer の設定
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.1)

    # nn -> pl: train 用の DataLoader の設定
    # @pl.data_loader
    def train_dataloader(self):
        return torch.utils.data.DataLoader(train, self.batch_size, shuffle=True)

    # nn -> pl: 学習データに対する処理
    def training_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        results = {'loss': loss}
        return results

### モデルの学習

In [54]:
from pytorch_lightning import Trainer

In [55]:
# 乱数のシード固定
torch.manual_seed(0)

# インスタンス化
net = Net()

In [56]:
# 学習用に用いるクラスの Trainer をインスタンス化
trainer = Trainer()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [57]:
# Trainer によるモデルの学習
trainer.fit(net)

C:\Users\ytchi\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\loops\utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
Missing logger folder: c:\Users\ytchi\Development\Python\work\lightning_logs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 55    
1 | fc2  | Linear | 18    
--------------------------------
73        Trainable params
0         Non-trainable params
73        Total params
0.000     Total estimated model params size (MB)
c:\Users\ytchi\anaconda3\envs\py37\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Epoch 0:  91%|█████████ | 10/11 [00:00<00:00, 65.03it/s, loss=4.47, v_num=0]

C:\Users\ytchi\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\ytchi\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\trainer.py:1600: PossibleUserWarning: The number of training batches (11) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 999: 100%|██████████| 11/11 [00:00<00:00, 75.16it/s, loss=1.1, v_num=0]  

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|██████████| 11/11 [00:00<00:00, 67.15it/s, loss=1.1, v_num=0]


In [59]:
# Trainer の引数の中でも代表的なものを紹介しておきます。

# 引数名                  デフォルトの値    説明
# show_progress_bar	    True            学習時の進捗を標準出力
# max_epochs	            1000            学習時の最大エポック数
# min_epochs	            1               学習時の最小エポック数
# train_percent_check	    1.0             学習データに対する確認の比率 (%)
# val_percent_check	    1.0             検証データに対する確認の比率 (%)
# test_percent_check	    1.0             テストデータに対する確認の比率 (%)
# early_stop_callback	    False           早期終了の使用の有無
# gpus	                None            使用するGPUの数
# distributed_backend	    None            分散学習の方法
# 抑えておくべき点として、デフォルトでは early_stop_callback=False であるため、
# 早期終了 (early stopping) が適用されていない。
# 早期終了とはある計測する指標に対して学習によって変化がなくなった場合に終了する方法。
# 基本的には、検証データに対する目的関数の値もしくは正解率などが指標として採用される。
# そのため、エポックの数は max_nb_epochs で最大の数を指定する程度で、
# あとは早期終了を有効にし、不要な学習は打ち切ることにすることも多い。

### 検証データの追加

In [60]:
# validation_step は検証データに対する各イテレーションごとの結果
# validation_end はエポック毎にその結果を集計
# 検証データやテストデータに対する計算の場合には、
# torch.no_grad() を用いて勾配情報を持たないようにしていたが、
# そういった処理も PyTorch Lightning 側で設定されている。

In [61]:
class Net(pl.LightningModule):
    # nn -> pl: バッチサイズ等を引数に指定する
    def __init__(self, input_size=10, hidden_size=5, output_size=3, batch_size=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size

    # nn -> pl: 変更なし
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

    # nn -> pl: 目的関数の設定
    def lossfun(self, y, t):
        return F.cross_entropy(y, t)
    
    # nn -> pl: optimizer の設定
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.1)

    # nn -> pl: train 用の DataLoader の設定
    # @pl.data_loader
    def train_dataloader(self):
        return torch.utils.data.DataLoader(train, self.batch_size, shuffle=True)

    # nn -> pl: 学習データに対する処理
    def training_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        results = {'loss': loss}
        return results

    # nn -> pl: 検証用データセットの設定
    # @pl.data_loader
    def val_dataloader(self):
        return torch.utils.data.DataLoader(val, self.batch_size)
    
    # nn -> pl: 検証データに対するイテレーションごとの処理
    def validation_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        
        # 正解率の算出
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'val_loss': loss, 'val_acc': acc}
        return results
    
    # nn -> pl: 検証データに対するエポックごとの処理
    def validation_end(self, outputs):
        # 各イテレーションごとに得られた値をまとめて、平均値の取得
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc  =torch.stack([x['val_acc'] for x in outputs]).mean()
        results = {'val_loss': avg_loss, 'val_acc': avg_acc}
        return results

In [62]:
torch.manual_seed(0)

net = Net()
trainer = Trainer()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [64]:
# モデルの学習
trainer.fit(net)

C:\Users\ytchi\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:612: UserWarning: Checkpoint directory c:\Users\ytchi\Development\Python\work\lightning_logs\version_1\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 55    
1 | fc2  | Linear | 18    
--------------------------------
73        Trainable params
0         Non-trainable params
73        Total params
0.000     Total estimated model params size (MB)


Epoch 138:  73%|███████▎  | 11/15 [00:00<00:00, 82.00it/s, loss=1.1, v_num=1] 

C:\Users\ytchi\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\ytchi\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\ytchi\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\trainer.py:1600: PossibleUserWarning: The number of training batches (11) is smaller than the

Epoch 999: 100%|██████████| 15/15 [00:00<00:00, 90.68it/s, loss=1.09, v_num=1] 

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|██████████| 15/15 [00:00<00:00, 85.91it/s, loss=1.09, v_num=1]


### テストデータを追加

In [65]:
# 検証データと同様に、テストデータに対する結果が得られるようにクラスにメソッドを追加

In [66]:
class Net(pl.LightningModule):
    # nn -> pl: バッチサイズ等を引数に指定する
    def __init__(self, input_size=10, hidden_size=5, output_size=3, batch_size=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size

    # nn -> pl: 変更なし
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

    # nn -> pl: 目的関数の設定
    def lossfun(self, y, t):
        return F.cross_entropy(y, t)
    
    # nn -> pl: optimizer の設定
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.1)

    # nn -> pl: train 用の DataLoader の設定
    # @pl.data_loader
    def train_dataloader(self):
        return torch.utils.data.DataLoader(train, self.batch_size, shuffle=True)

    # nn -> pl: 学習データに対する処理
    def training_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        results = {'loss': loss}
        return results

    # nn -> pl: 検証用データセットの設定
    # @pl.data_loader
    def val_dataloader(self):
        return torch.utils.data.DataLoader(val, self.batch_size)
    
    # nn -> pl: 検証データに対するイテレーションごとの処理
    def validation_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        
        # 正解率の算出
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'val_loss': loss, 'val_acc': acc}
        return results
    
    # nn -> pl: 検証データに対するエポックごとの処理
    def validation_end(self, outputs):
        # 各イテレーションごとに得られた値をまとめて、平均値の取得
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc  =torch.stack([x['val_acc'] for x in outputs]).mean()
        results = {'val_loss': avg_loss, 'val_acc': avg_acc}
        return results

    # nn -> pl: テストデータセットの設定
    # @pl.data_loader
    def test_dataloader(self):
        return torch.utils.data.DataLoader(test, self.batch_size)
    
    # nn -> pl: テストデータに対するイテレーションごとの処理
    def test_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'test_loss': loss, 'test_acc': acc}
        return results
    
    # nn -> pl: テストデータに対するエポックごとの処理
    def test_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        results = {'test_loss': avg_loss, 'test_acc': avg_acc}
        return results

In [67]:
# 学習に関する一連の流れを実行
torch.manual_seed(0)

net = Net()
trainer = Trainer()

trainer.fit(net)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\ytchi\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\loops\utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 55    
1 | fc2  | Linear | 18    
--------------------------------
73        Trainable params
0         Non-trainable params
73        Total params
0.000     Total estimated model params size (MB)


Epoch 999: 100%|██████████| 15/15 [00:00<00:00, 67.50it/s, loss=1.1, v_num=2]  

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|██████████| 15/15 [00:00<00:00, 62.78it/s, loss=1.1, v_num=2]


In [68]:
# 最終的な結果は trainer のcallback_metrics に格納される。
# テストデータに対する結果を検証する場合は trainer.test() メソッドが用意されており、
# 実行すると検証用データセットに対する結果とテスト用データセットに対する結果の両方が確認できる。

# テストデータに対する結果を検証する場合は test メソッドが用意されており、
# 検証用データセットに対する結果とテスト用データセットに対する結果が格納されている。

In [83]:
# テストデータに対する処理の実行（test_step と test_end）
trainer.test()

Restoring states from the checkpoint path at c:\Users\ytchi\Development\Python\work\lightning_logs\version_4\checkpoints\epoch=298-step=3289.ckpt
Loaded model weights from checkpoint at c:\Users\ytchi\Development\Python\work\lightning_logs\version_4\checkpoints\epoch=298-step=3289.ckpt


Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 106.98it/s]


[{}]

In [86]:
#  テストデータに対する結果の確認
trainer.callback_metrics

{}

### 可読性と汎用性を向上

In [71]:
# 学習データ、検証データ、テストデータのそれぞれに対する処理を
# TrainNet、ValidationNet、TestNet のクラスにそれぞれ記述し、
# それらを継承した Net に変化のある部分を記述

# このように記述を行うと、Net でモデルの構造を記述するだけで良く、可読性が高まる
# また、各種のデータに対する処理は毎回同じものを使うことができ、汎用性も高まる

In [72]:
# 学習データに対する処理
class TrainNet(pl.LightningModule):

    # @pl.data_loader
    def train_dataloader(self):
        return torch.utils.data.DataLoader(train, self.batch_size, shuffle=True)

    def training_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        results = {'loss': loss}
        return results

In [73]:
# 検証データに対する処理
class ValidationNet(pl.LightningModule):

    # @pl.data_loader
    def val_dataloader(self):
        return torch.utils.data.DataLoader(val, self.batch_size)

    def validation_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'val_loss': loss, 'val_acc': acc}
        return results

    def validation_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        results = {'val_loss': avg_loss, 'val_acc': avg_acc}
        return results

In [74]:
# テストデータに対する処理
class TestNet(pl.LightningModule):

    # @pl.data_loader
    def test_dataloader(self):
        return torch.utils.data.DataLoader(test, self.batch_size)

    def test_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'test_loss': loss, 'test_acc': acc}
        return results

    def test_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['test_acc'] for x in outputs]).mean()
        results = {'test_loss': avg_loss, 'test_acc': avg_acc}
        return results

In [75]:
# 学習データ、検証データ、テストデータへの処理を継承したクラス
class Net(TrainNet, ValidationNet, TestNet):
    
    def __init__(self, input_size=10, hidden_size=5, output_size=3, batch_size=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

    def lossfun(self, y, t):
        return F.cross_entropy(y, t)

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.1)

In [76]:
# 学習に関する一連の流れを実行
torch.manual_seed(0)

net = Net()
trainer = Trainer()

trainer.fit(net)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 55    
1 | fc2  | Linear | 18    
--------------------------------
73        Trainable params
0         Non-trainable params
73        Total params
0.000     Total estimated model params size (MB)


Epoch 999: 100%|██████████| 15/15 [00:00<00:00, 76.49it/s, loss=1.1, v_num=3]  

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|██████████| 15/15 [00:00<00:00, 71.47it/s, loss=1.1, v_num=3]


### 予測精度向上させるテクニック

In [77]:
# バッチノーマリゼーション (Batch Normalization)
# ミニバッチごとに平均 μ と 標準偏差 σ を求め、
# 平均 β、標準偏差 α となるように変換を行う。
# 必ずしも平均 0、標準偏差 1 が良いとは限らないため、
# 標準化した後に少し値を変換させることで完全に分布を制限してしまうことを避けている。

In [79]:
# 実装としては、各バッチ毎に平均 μ と標準偏差 σ を定めて標準化を行うという簡単な手法

In [81]:
# 学習データ、検証データ、テストデータへの処理を継承したクラス
class Net(TrainNet, ValidationNet, TestNet):
    
    def __init__(self, input_size=10, hidden_size=5, output_size=3, batch_size=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        self.bn = nn.BatchNorm1d(input_size) # 追加

    def forward(self, x):
        x = self.bn(x) # 追加
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

    def lossfun(self, y, t):
        return F.cross_entropy(y, t)

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.1)

In [87]:
# 学習に関する一連の流れを実行
torch.manual_seed(0)

net = Net()
trainer = Trainer()

trainer.fit(net)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 55    
1 | fc2  | Linear      | 18    
2 | bn   | BatchNorm1d | 20    
-------------------------------------
93        Trainable params
0         Non-trainable params
93        Total params
0.000     Total estimated model params size (MB)


Epoch 999: 100%|██████████| 15/15 [00:00<00:00, 92.62it/s, loss=0.0283, v_num=5] 

`Trainer.fit` stopped: `max_epochs=1000` reached.


Epoch 999: 100%|██████████| 15/15 [00:00<00:00, 86.43it/s, loss=0.0283, v_num=5]


In [89]:
# テストデータに対する処理の実行（test_step と test_end）
trainer.test()

Restoring states from the checkpoint path at c:\Users\ytchi\Development\Python\work\lightning_logs\version_5\checkpoints\epoch=999-step=11000.ckpt
Loaded model weights from checkpoint at c:\Users\ytchi\Development\Python\work\lightning_logs\version_5\checkpoints\epoch=999-step=11000.ckpt


Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 168.83it/s]


[{}]

In [90]:
# 結果の確認
trainer.callback_metrics

{}

### 学習済みモデルの保存から推論

In [91]:
# モデルの重み
net.state_dict()

OrderedDict([('fc1.weight',
              tensor([[-0.6206, -0.1084,  0.8248, -1.4392, -0.9782,  1.6249,  0.5771,  0.2370,
                        0.3248, -1.5282],
                      [-1.0523, -0.8148, -0.0732,  1.1580,  0.9905, -1.0875, -0.7037,  0.1645,
                        1.4862, -0.0513],
                      [-1.1830, -0.1483,  0.5771,  0.0274,  0.0606, -0.3696, -0.4341, -0.1305,
                       -0.5154,  0.4619],
                      [-0.3722,  0.7309,  0.1979, -0.0273,  0.5360, -1.3404, -0.4401,  1.4830,
                        0.2171, -0.4927],
                      [ 0.5829, -0.4404,  0.1842,  0.3183,  0.0820, -1.2322,  0.1969, -0.0223,
                        0.1850, -0.4095]])),
             ('fc1.bias', tensor([0.2524, 0.4649, 0.3014, 0.2918, 0.8056])),
             ('fc2.weight',
              tensor([[-0.8988,  0.0971, -1.4651, -0.3207, -1.7956],
                      [ 1.1254,  1.0925,  0.4449, -0.5528,  1.1171],
                      [-0.3440, -0.9093, 

In [92]:
# 学習済みモデルを保存
torch.save(net.state_dict(), 'wine.pt')

### 学習済みモデルを使用した推論

In [93]:
# 学習済みモデルは単にファイルをロードするだけでなく、
# モデルの構造を明示しておき、そのモデルに対して、
# パラメータの値を当てはめながらロードしていくことになる
# 成功すれば <All keys matched successfully> とでるが
# パラメータ数などが合わなければ、失敗する

In [94]:
# インスタンス化
net = Net()

In [95]:
net.load_state_dict(torch.load('wine.pt'))

<All keys matched successfully>

### 予測値の計算

In [96]:
# 今回は新しいデータが無いので学習で使用したデータセットのいちばん最初のサンプルに対する予測値を計算
# 予測値の計算
y = net(x)[0]
y

tensor([ 4.5857, -2.4222, -2.4962], grad_fn=<SelectBackward0>)

In [97]:
# 予測ラベル
torch.argmax(y)

tensor(0)

In [98]:
# 目的変数
t[0]

tensor(0)